In [1]:
%load_ext autoreload
%autoreload 2
%autosave 60
%matplotlib inline
from ivanocode.ivanocommon import *
import pandas as pd

Autosaving every 60 seconds


In [2]:
raw = 'raw'
submissions = 'submissions'

In [3]:
df_sample_submission = pd.read_csv(f'{raw}/sample_submission.csv')
df_sample_submission.columns

Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')

In [4]:
df_sales_train_melt = pd.read_feather('df_sales_train_melt.feather')

In [5]:
df_sales_train_melt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,daily_avg_count,monthly_avg_count,day_date,day_date_str,month_id,month,year,date,wm_yr_wk,sell_price,sell_price_flip_count
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0.313643,9.409305,2011-01-29,2011-01-29,1,1,2011,2011-01-29,11101,NaN,1


In [6]:
days = df_sales_train_melt['day_id'].unique()
days[0], days[-1]

(1, 1913)

In [7]:
# F1 = d_1914, ... F28 = d_1941

In [8]:
query = ("id == 'HOBBIES_1_001_CA_1_validation' or "+
         "id == 'HOBBIES_1_002_CA_1_validation' or "+
         "id == 'HOBBIES_1_004_CA_1_validation'")
subset = df_sales_train_melt.query(query).copy()

In [9]:
%%time
min_day = 1885
df_submission_melt = (df_sales_train_melt
    .groupby('id', as_index=False)
    .tail(28)
)

df_submission = (df_submission_melt
    .pivot(index='id', columns='day_id', values='sales')
    .rename(lambda x: f'F{x-min_day}', axis=1)
    .reset_index()
    .sort_values('id')
)

CPU times: user 5.15 s, sys: 1.6 s, total: 6.75 s
Wall time: 6.87 s


In [10]:
df_train = df_sales_train_melt.query('day_id < 1886').copy()

In [11]:
df_pred = df_train.query(f'day_id >= {1886-28} & day_id <= {1913-28}').copy()

In [12]:
df_valid = df_sales_train_melt.query('day_id >= 1886 & day_id <= 1913').copy()

In [47]:
aggregation_levels = {
    11: ['item_id', 'state_id'],
    10: ['item_id'],
    9:  ['store_id', 'dept_id'],
    8:  ['store_id', 'cat_id'],
    7:  ['state_id', 'dept_id'],
    6:  ['state_id', 'cat_id'],
    5:  ['dept_id'],
    4:  ['cat_id'],
    3:  ['store_id'],
    2:  ['state_id'],
    1:  [],
}
agg_level_expected_counts = {
    11: 9147,
    10: 3049,
    9:  70,
    8:  30,
    7:  21,
    6:  9,
    5:  7,
    4:  3,
    3:  10,
    2:  3
}
def with_aggregates(df):
    result = [df]
    for grp_id, grp_fields in aggregation_levels.items():
        grp_11 = df.groupby(grp_fields + ['day_id'], as_index=False)['sales'].sum()
        grp_11['agg_level'] = grp_id
        grp_11['id'] = str(grp_id)
        if len(grp_fields) > 0:
            # Note to self: categoricals require cast
            for col in grp_fields:
                grp_11['id'] = grp_11['id'].str.cat(grp_11[col], sep=':')

        result.append(grp_11)

    return pd.concat(result, sort=False)

df_valid_w_aggs = with_aggregates(df_valid)
agg_series_counts = (df_valid_w_aggs.query('day_id == 1886')
    [['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'agg_level', 'state_id']]
    .groupby(['agg_level'], as_index=False)
    .count()
)

In [51]:
df_valid_w_aggs.query('day_id==1909')['id'].count()

42840

In [27]:
grp_fields=['state_id', 'cat_id']
grp_11 = df_valid.groupby(grp_fields + ['day_id'], as_index=False)['sales'].sum()
grp_11['id'] = grp_11[grp_fields[0]]
for col in grp_fields[1:]:
    grp_11['id'] = grp_11['id'].str.cat(grp_11[col], sep='_')
grp_11.head()

,state_id,cat_id,day_id,sales,id
0,CA,FOODS,1886,9821,CA_FOODS
1,CA,FOODS,1887,9845,CA_FOODS
2,CA,FOODS,1888,9165,CA_FOODS
3,CA,FOODS,1889,9941,CA_FOODS
4,CA,FOODS,1890,11891,CA_FOODS


In [52]:
%%time
for level, expected_count in agg_level_expected_counts.items():
    actual_count = (agg_series_counts
         .query(f'agg_level == {level}')
         .drop('agg_level', axis=1)
         .max(axis=1)
         .values[0]
    )
    assert expected_count == actual_count, f"Expected {expected_count} at level {level}, got: {actual_count}"

CPU times: user 29.9 ms, sys: 1.35 ms, total: 31.3 ms
Wall time: 30.4 ms


In [53]:
# grp_11['id'] = grp_11.sum(axis=1) # fishy, ints and not they sorta collide
"""
>>> df_valid_w_aggs.query('day_id == 1886')['id'].value_counts()[:3]
1897    3041
1898    1952
1899    1394
Name: id, dtype: int64
"""

"\n>>> df_valid_w_aggs.query('day_id == 1886')['id'].value_counts()[:3]\n1897    3041\n1898    1952\n1899    1394\nName: id, dtype: int64\n"

In [54]:
# TODO:
# pre-aggregate series
# add weight per sales volumes
# calculate rmssse
# calculate wrmsse
# ...
# Profit!

In [55]:
eval_rows = df_sample_submission[df_sample_submission['id'].str.contains('_evaluation')]

In [56]:
(pd.concat([df_submission, eval_rows])
    .to_csv(f'{submissions}/0200-naive.csv', index=False)
)

In [57]:
!head -n2 {submissions}/0200-naive.csv

id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
FOODS_1_001_CA_1_validation,2,1,1,0,4,0,0,4,1,3,0,1,0,2,2,0,1,1,0,2,0,4,1,1,0,1,1,0


In [58]:
!head -n2 {raw}/sample_submission.csv

id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
!wc -l {submissions}/0200-naive.csv

   60981 submissions/0200-naive.csv


In [60]:
!wc -l {raw}/sample_submission.csv

   60981 raw/sample_submission.csv


In [25]:
!open {submissions}

In [26]:
!open https://www.kaggle.com/c/m5-forecasting-accuracy/submit